# Sample Template

Here is some sample text, and a first block of code

In [ ]:
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'  # always print last expr.

In [ ]:
import re

In [ ]:
def ignore_subgroups(pattern: str | re.Pattern, /) -> str:
    """Ignore all named groups in the given pattern."""
    return re.sub(r"\(\?P<[^>]+>", r"(?:", pattern)


# https://peps.python.org/pep-0440/#appendix-b-parsing-version-strings-with-regular-expressions
VERSION = r"""(?ix:                                       # case-insensitive, verbose
    v?(?:
        (?:(?P<epoch>[0-9]+)!)?                           # epoch
        (?P<release>[0-9]+(?:\.[0-9]+)*)                  # release segment
        (?P<pre>                                          # pre-release
            [-_.]?
            (?P<pre_l>(?:a|b|c|rc|alpha|beta|pre|preview))
            [-_.]?
            (?P<pre_n>[0-9]+)?
        )?
        (?P<post>                                         # post release
            (?:-(?P<post_n1>[0-9]+))
            |
            (?:
                [-_.]?
                (?P<post_l>post|rev|r)
                [-_.]?
                (?P<post_n2>[0-9]+)?
            )
        )?
        (?P<dev>                                          # dev release
            [-_.]?
            (?P<dev_l>dev)
            [-_.]?
            (?P<dev_n>[0-9]+)?
        )?
    )
    (?:\+(?P<local>[a-z0-9]+(?:[-_.][a-z0-9]+)*))?        # local version
)"""
r"^([1-9][0-9]*!)?(0|[1-9][0-9]*)(\.(0|[1-9][0-9]*))*((a|b|rc)(0|[1-9][0-9]*))?(\.post(0|[1-9][0-9]*))?(\.dev(0|[1-9][0-9]*))?$"
VERSION_GROUP = rf"""(?P<version>{ignore_subgroups(VERSION)})"""
VERSION_REGEX: re.Pattern = re.compile(VERSION_GROUP)

VERSION_REGEX.groups

In [ ]:
NAME = re.compile(r"""\b[a-zA-Z0-9](?:[a-zA-Z0-9._-]*[a-zA-Z0-9])?\b""")
NAME_GROUP = rf"""(?P<name>{NAME.pattern})"""
NAME_REGEX = re.compile(NAME_GROUP)
assert NAME_REGEX.groups == 1, f"NAME_REGEX has {NAME_REGEX.groups} groups."

In [ ]:
import pyarrow as pa

pa_x = pa.scalar(None, type=pa.int64())
pa_y = pa.compute.greater(pa_x, None)
result = pa_y.as_py()  # None

py_x = pa_x.as_py()
py_y = (
    py_x > 0
)  # TypeError: '>' not supported between instances of 'NoneType' and 'int'

In [ ]:
re.compile(r"(?ix:\w+)").groups

In [ ]:
dir(VaERSION_REGEX)

In [ ]:
# https://peps.python.org/pep-0508/#names
# NOTE: we modify this regex a bit to allow to match inside context
NAME = re.compile(r"""\b[a-zA-Z0-9](?:[a-zA-Z0-9._-]*[a-zA-Z0-9])?\b""")
NAME_GROUP = rf"""(?P<name>{NAME})"""
NAME_REGEX = re.compile(NAME_GROUP)
assert NAME_REGEX.groups == 1

# NOTE: to get a list of extras, match NAME_PATTERN with EXTRAS_PATTERN
EXTRAS = rf"""\[\s*(?:{NAME})(?:\s*,{NAME})*\s*\]"""
EXTRAS_GROUP = rf"""(?P<extras>{EXTRAS})"""
EXTRAS_REGEX = re.compile(EXTRAS_GROUP)
assert EXTRAS_REGEX.groups == 1

PROJECT_DEP = rf"""["']{NAME_GROUP}{EXTRAS}?\s*>=\s*{VERSION_GROUP}"""
PROJECT_DEP_GROUP = rf"""(?P<DEPENDENCY>{PROJECT_DEP})"""
PROJECT_DEP_REGEX = re.compile(PROJECT_DEP_GROUP)

In [ ]:
NAME = re.compile(r"""\b[a-zA-Z0-9](?:[a-zA-Z0-9._-]*[a-zA-Z0-9])?\b""")
NAME_GROUP = re.compile(rf"""(?P<name>{NAME.pattern})""")
NAME_REGEX = re.compile(NAME_GROUP)

In [ ]:
PROJECT_DEP = rf"""["']{NAME_GROUP}{EXTRAS}?\s*>=\s*{VERSION_GROUP}"""
PROJECT_DEP_GROUP = rf"""(?P<DEPENDENCY>{PROJECT_DEP})"""
PROJECT_DEP_REGEX = re.compile(PROJECT_DEP_GROUP)

In [ ]:
import re


def is_canonical(version):
    return (
        re.match(
            r"^([1-9][0-9]*!)?(0|[1-9][0-9]*)(\.(0|[1-9][0-9]*))*((a|b|rc)(0|[1-9][0-9]*))?(\.post(0|[1-9][0-9]*))?(\.dev(0|[1-9][0-9]*))?$",
            version,
        )
        is not None
    )

In [ ]:
VERSION_PATTERN = r"""
    v?
    (?:
        (?:(?P<epoch>[0-9]+)!)?                           # epoch
        (?P<release>[0-9]+(?:\.[0-9]+)*)                  # release segment
        (?P<pre>                                          # pre-release
            [-_\.]?
            (?P<pre_l>(a|b|c|rc|alpha|beta|pre|preview))
            [-_\.]?
            (?P<pre_n>[0-9]+)?
        )?
        (?P<post>                                         # post release
            (?:-(?P<post_n1>[0-9]+))
            |
            (?:
                [-_\.]?
                (?P<post_l>post|rev|r)
                [-_\.]?
                (?P<post_n2>[0-9]+)?
            )
        )?
        (?P<dev>                                          # dev release
            [-_\.]?
            (?P<dev_l>dev)
            [-_\.]?
            (?P<dev_n>[0-9]+)?
        )?
    )
    (?:\+(?P<local>[a-z0-9]+(?:[-_\.][a-z0-9]+)*))?       # local version
"""
_regex = re.compile(VERSION_PATTERN, re.VERBOSE | re.IGNORECASE)


def is_canonical2(version):
    return re.match(_regex, version) is not None


is_canonical2("0.1.23")

In [ ]:
VERSION = r"""(?ix:                                       # case-insensitive, verbose
    v?(?:
        (?:(?P<epoch>[0-9]+)!)?                           # epoch
        (?P<release>[0-9]+(?:\.[0-9]+)*)                  # release segment
        (?P<pre>                                          # pre-release
            [-_\.]?
            (?P<pre_l>(a|b|c|rc|alpha|beta|pre|preview))
            [-_\.]?
            (?P<pre_n>[0-9]+)?
        )?
        (?P<post>                                         # post release
            (?:-(?P<post_n1>[0-9]+))
            |
            (?:
                [-_\.]?
                (?P<post_l>post|rev|r)
                [-_\.]?
                (?P<post_n2>[0-9]+)?
            )
        )?
        (?P<dev>                                          # dev release
            [-_\.]?
            (?P<dev_l>dev)
            [-_\.]?
            (?P<dev_n>[0-9]+)?
        )?
    )
    (?:\+(?P<local>[a-z0-9]+(?:[-_\.][a-z0-9]+)*))?        # local version
)"""
VERSION_GROUP = rf"""(?P<version>{VERSION})"""
VERSION_REGEX = re.compile(VERSION)


def is_canonical3(version):
    return re.match(VERSION_REGEX, version) is not None


is_canonical3("aga.aga.aga")

In [ ]:
VERSION_REGEX.match("0.1.23")

In [ ]:
VERSION_REGEX.match(".1.23")

In [ ]:
import re

In [ ]:
is_canonical2("0.1.23")

In [ ]:
tests = r"""
black[ jupyterlab , d ]
pandas[numpy]
pkg[   ext1 , ext2, ext3, ext4, ext5]
a "witch" and her "broom" is one
"""

In [ ]:
pattern = r'"[^\"]+"'
matches = re.findall(pattern, tests)

In [ ]:
name = r"(\b[\w-]+\b)"
extras = f"\[(?:[,\s*]?{name}[,\s*]?)*\]"

pattern = r"\[\s*(?:(?:\b[\w-]+\b\s*,\s*)*(\b[\w-]+\b)(?:\s*,\s*\b[\w-]+\b)*)+\s*\]"
regex = re.compile(pattern, flags=re.VERBOSE)
# matches = list(re.finditer(pattern, tests))
matches = re.findall(pattern, tests)

In [ ]:
?re.compile

In [ ]:
# https://peps.python.org/pep-0440/#appendix-b-parsing-version-strings-with-regular-expressions
VERSION_PATTERN = """
v?
(?:
    (?:(?P<epoch>[0-9]+)!)?                           # epoch
    (?P<release>[0-9]+(?:\.[0-9]+)*)                  # release segment
    (?P<pre>                                          # pre-release
        [-_\.]?
        (?P<pre_l>(a|b|c|rc|alpha|beta|pre|preview))
        [-_\.]?
        (?P<pre_n>[0-9]+)?
    )?
    (?P<post>                                         # post release
        (?:-(?P<post_n1>[0-9]+))
        |
        (?:
            [-_\.]?
            (?P<post_l>post|rev|r)
            [-_\.]?
            (?P<post_n2>[0-9]+)?
        )
    )?
    (?P<dev>                                          # dev release
        [-_\.]?
        (?P<dev_l>dev)
        [-_\.]?
        (?P<dev_n>[0-9]+)?
    )?
)
(?:\+(?P<local>[a-z0-9]+(?:[-_\.][a-z0-9]+)*))?       # local version
"""

VERSION_REGEX: re.Pattern = re.compile(
    f"""(?P<version>{VERSION_PATTERN})""",
    re.VERBOSE | re.IGNORECASE,
)

# https://peps.python.org/pep-0508/#names
NAME_PATTERN = r"[A-Z0-9]|[A-Z0-9][A-Z0-9._-]*[A-Z0-9]"
NAME_REGEX = re.compile(
    f"(?P<name>{NAME_PATTERN})",
    re.IGNORECASE,
)

EXTRAS_PATTERN = r"\n\[\s*(?:{NAME_PATTERN}\s*,\s*)*{NAME_PATTERN}\s*\]"
EXTRAS_REGEX = re.compile(
    f"(?P<extras>{EXTRAS_PATTERN})",
    re.IGNORECASE,
)

In [ ]:
EXTRAS_PATTERN.format(NAME_PATTERN=NAME_PATTERN)

In [ ]:
EXTRAS_PATTERN = f"\n\[\s*(?:{NAME_PATTERN}\s*,\s*)*{NAME_PATTERN}\s*\]"

In [ ]:
rf" \{{"

In [ ]:
r" \{"

In [ ]:
EXTRAS_PATTERN

In [ ]:
\[\s*  (  (\b[\w-]+\b)  (\s*,\s*)?  )*  \s*\]